# Real Rankine Cycle with Feedwater Heater
A steam power plant operates on a cycle with pressures and temperatures as given below. The efficiency of the turbine is 86%, and the efficiency of the pumps are 80%. Determine the thermal efficiency of this cycle.

![real rankine with feedwater heater](Images/real_rankine_with_fwh.png)

Given variables

In [1]:
p1 = 0.01 # 10 kPa
T1 = 42 + 273.15 # kelvins
p2 = 0.4 # 400 kPa
p3 = p2
p8 = p2
p4 = 5 # MPa
p5 = 4.8 # MPa
T5 = 130 + 273.15 # kelvins
p6 = 4 # MPa
T6 = 400 + 273.15 # kelvins
p7 = 3.8 # MPa
T7 = 380 + 273.15 # kelvins
p9 = p1

eta_turbine = 0.86
eta_pump = 0.8
# eta_turbine = 1 # to see how ideal cycle efficiency compares
# eta_pump = 1 # to see how ideal cycle efficiency compares

Analyze the low pressure pump

In [2]:
from iapws import IAPWS97
sat_liquid1 = IAPWS97(P=p1,x=0)
v1 = sat_liquid1.v
h1 = sat_liquid1.h

w_pump1 = v1*(p2-p1) / eta_pump * 1000
print(f"The work of the low pressure pump is: {round(float(w_pump1),1)} kJ/kg")

The work of the low pressure pump is: 0.5 kJ/kg


In [3]:
h2 = h1 + w_pump1
print(f"The enthalpy at point 2 is: {round(float(h2),1)} kJ/kg")

The enthalpy at point 2 is: 192.3 kJ/kg


Analyze the turbine

In [4]:
vapor7 = IAPWS97(P=p7,T=T7)
s7 = vapor7.s
s8 = s7
s9 = s7
h7 = vapor7.h

from custom_thermo import custom_thermo as ct
x8 = ct.find_quality(P=p8,s=s8)
x9 = ct.find_quality(P=p9,s=s9)

vapor8 = IAPWS97(P=p8,x=x8)
h8 = vapor8.h

# w_turbine = h7 - (y*h8 + (1-y)*h9)
print(f"The quality of the line from the turbine to the FWH is: {round(x8,4)}")
print(f"The quality after the turbine is: {round(x9,4)}")
# print(f"The work of the turbine is: {round(float(w_turbine),1)} kJ/kg")

The quality of the line from the turbine to the FWH is: 0.9672
The quality after the turbine is: 0.8105


From conservation of energy for the feedwater heater:
$$m_7\cdot h_3=y m_7\cdot h_8+(1-y) m_7\cdot h_2$$
$$h_3=y(h_8)+(1-y)h_2$$
This can be re-arranged to solve y explicitly:
$$y=\frac{h_3-h_2}{h_8-h_2}$$

In [5]:
vapor9 = IAPWS97(P=p9,x=x9)
h9 = vapor9.h
liquid3 = IAPWS97(P=p3,x=0)
h3 = liquid3.h

y = (h3-h2)/(h8-h2)
w_turbine = (h7 - (h8*y + h9*(1-y)))*eta_turbine

print(f"The mass fraction of fluid diverted to the FWH from the turbine is y = {round(float(y),4)}")
print(f"The work output of the turbine is: {round(float(w_turbine),1)} kJ/kg")

The mass fraction of fluid diverted to the FWH from the turbine is y = 0.1666
The work output of the turbine is: 817.4 kJ/kg


Analyze the high pressure pump

In [6]:
v3 = liquid3.v

w_pump2 = v3*(p4-p3) / eta_pump * 1000
h4 = h3 + w_pump2
print(f"The work of the high pressure pump is: {round(float(w_pump2),1)} kJ/kg")

The work of the high pressure pump is: 6.2 kJ/kg


Analyze the boiler

In [7]:
liquid5 = IAPWS97(T=T5,x=0)
h5 = liquid5.h
vapor6 = IAPWS97(T=T6,P=p6)
h6 = vapor6.h

q_H = h6 - h5
print(f"The heat input of the boiler is: {round(float(q_H),1)} kJ/kg")

The heat input of the boiler is: 2668.0 kJ/kg


Calculate the thermal efficiency of the cycle

In [8]:
eta_th = (w_turbine - w_pump1 - w_pump2) / q_H * 100

print(f"The thermal efficiency of the cycle is: {round(eta_th,1)}%")

The thermal efficiency of the cycle is: 30.4%
